In [1]:
import sys
import os

# append the path of the parent directory
sys.path.append("../src")
os.path.abspath(os.getcwd())

'/home/michel/Documents/BPO_Batching/notebooks'

In [2]:
import collections
import numpy as np
from ilp_policy_non_assign_2 import *
from hungarian_policy import *

In [3]:
class Task:
    def __init__(self, name, task_type):
        self.name = name
        self.task_type = task_type

    def __str__(self):
        return self.name

    def __repr__(self):
        return self.name

In [37]:
t1 = Task('t1', 't1')
t2 = Task('t2', 't2')

unassigned_tasks = [t1, t2]
initial_available_resources = ['Ra', 'Rb']
working_resources = {}
resource_pool = {
    't1' : ['Ra', 'Rb'],
    't2' : ['Ra', 'Rc'],
}
trds = {
    (t1, 'Ra') : 3,
    #(t1, 'Rb') : 10,
    (t1, 'Rb') : 15,
    (t1, 'Rd') : 21,

    (t2, 'Ra') : 7,
    (t2, 'Rc') : 7,
    (t2, 'Rd') : 22,
}
task_durations = collections.defaultdict(list)
for (t, r), trd in trds.items():
    task_durations[t].append(trd)
mean_durations = {k: np.mean(v) for k, v in task_durations.items()}

In [38]:
print(task_durations)
print(mean_durations)

defaultdict(<class 'list'>, {t1: [3, 15, 21], t2: [7, 7, 22]})
{t1: 13.0, t2: 12.0}


In [6]:
print(unassigned_tasks, initial_available_resources, resource_pool, trds, {}, {}, mean_durations, working_resources, 0)

[t1, t2] ['Ra', 'Rb'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {(t1, 'Ra'): 3, (t1, 'Rb'): 15, (t2, 'Ra'): 7, (t2, 'Rc'): 7, (t2, 'Rd'): 50} {} {} {t1: 9.0, t2: 21.333333333333332} {} 0


In [7]:
def do_allocation(policy, unassigned_tasks, available_resources, resource_pool, trds, mean_durations, working_resources, time):
    allocation = policy.allocate(unassigned_tasks, available_resources, resource_pool, trds, {}, {}, mean_durations, working_resources, time)
    return allocation

def add_working_resources(time, trds, allocation, available_resources, working_resources):
    new_available_resources = available_resources.copy()
    new_working_resources = working_resources.copy()
    for t, r in allocation:
        new_working_resources[r] = (time, trds[(t,r)])
        new_available_resources.remove(r)
    return new_available_resources, new_working_resources

def remove_working_resources(time, available_resources, working_resources):
    new_available_resources = available_resources.copy()
    new_working_resources = working_resources.copy()
    for r, ts in working_resources.items():
        if sum(ts) <= time:
            del new_working_resources[r]
            new_available_resources.append(r)
    return new_available_resources, new_working_resources

def remove_unassigned_tasks(allocation, unassigned_tasks):
    new_unassigned_tasks = unassigned_tasks.copy()
    for t,r in allocation:
        new_unassigned_tasks.remove(t)
    return new_unassigned_tasks

def remove_trds(allocation, trds):
    new_trds = trds.copy()
    for t,r in allocation:
        for tt,rr in trds.keys():
            if t==tt:
                del new_trds[(tt,rr)]
    return new_trds

In [8]:
working_resources.values()

dict_values([])

In [9]:
next_event = lambda working_resources : min(map(sum, working_resources.values())) if working_resources else np.inf


def do_simulation(policy, time, unassigned_tasks, available_resources, working_resources, trds, mean_durations, resource_pool):
    log = ''
    unassigned_tasks = unassigned_tasks.copy()
    available_resources = available_resources.copy()
    working_resources = working_resources.copy()
    trds = trds.copy()

    resource_rc_spawned = False
    
    while len(working_resources) or len(unassigned_tasks):
        log += str(time)+'\n'

        available_resources, working_resources = remove_working_resources(time, available_resources, working_resources)
        print(unassigned_tasks, available_resources, resource_pool, trds, mean_durations.copy(), working_resources, time)
        allocation = do_allocation(policy, unassigned_tasks, available_resources, resource_pool, trds, mean_durations.copy(), working_resources, time)
        log += 'Suggested allocation: ' + str(allocation)+'\n'
        
        real_allocation_dict = collections.defaultdict(list)
        for task, resource in allocation:
            real_allocation_dict[resource].append((trds[(task, resource)], task))
        allocation = []
        for resource, tasks in real_allocation_dict.items():
            allocation.append((sorted(tasks, key=lambda x : x[0])[0][1], resource))

        log += 'Real allocation: ' + str(allocation) + '\n'
        
        unassigned_tasks = remove_unassigned_tasks(allocation, unassigned_tasks)
        available_resources, working_resources = add_working_resources(time, trds, allocation, available_resources, working_resources)
        trds = remove_trds(allocation, trds)
        log += ', '.join(map(str, [unassigned_tasks, available_resources, trds, working_resources]))+'\n'

        time = next_event(working_resources)
        if not resource_rc_spawned:
            time = min(time, 2)
            #log += str(time)+
            if time == 2:
                log += 'Rc spawned\n'
                resource_rc_spawned = True
                available_resources.append('Rc')
        last_time = time
    log += '---------\n'

    return last_time, log


In [40]:
start_time, last_time = 0, 0
time = start_time
for i in [1]:#np.arange(-0.1, 8, 0.001):
    policy = HungarianMultiObjectivePolicy(1, 0, 0, i)
    time, log = do_simulation(policy, start_time, unassigned_tasks, initial_available_resources, working_resources, trds, mean_durations, resource_pool)
    if time != last_time:
        print('Delta:', str(i))
        print(log)
        last_time = time
        print('===========================================')

[t1, t2] ['Ra', 'Rb'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {(t1, 'Ra'): 3, (t1, 'Rb'): 15, (t1, 'Rd'): 21, (t2, 'Ra'): 7, (t2, 'Rc'): 7, (t2, 'Rd'): 22} {t1: 13.0, t2: 12.0} {} 0
[t2] ['Rb', 'Rc'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {(t2, 'Ra'): 7, (t2, 'Rc'): 7, (t2, 'Rd'): 22} {t1: 13.0, t2: 12.0} {'Ra': (0, 3)} 2
[] ['Rb', 'Ra'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {} {t1: 13.0, t2: 12.0} {'Rc': (2, 7)} 3
[] ['Rb', 'Ra', 'Rc'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {} {t1: 13.0, t2: 12.0} {} 9
Delta: 1
0
Suggested allocation: [(t1, 'Ra')]
Real allocation: [(t1, 'Ra')]
[t2], ['Rb'], {(t2, 'Ra'): 7, (t2, 'Rc'): 7, (t2, 'Rd'): 22}, {'Ra': (0, 3)}
Rc spawned
2
Suggested allocation: [(t2, 'Rc')]
Real allocation: [(t2, 'Rc')]
[], ['Rb'], {}, {'Ra': (0, 3), 'Rc': (2, 7)}
3
Suggested allocation: []
Real allocation: []
[], ['Rb', 'Ra'], {}, {'Rc': (2, 7)}
9
Suggested allocation: []
Real allocation: []
[], ['Rb', 'Ra', 'Rc'], {}, {}
---------



In [39]:
start_time, last_time = 0, 0
time = start_time
for i in [1]:

    policy = UnrelatedParallelMachinesSchedulingNonAssignPolicy2(1, 0, 0, i, 'first')
    time, log = do_simulation(policy, start_time, unassigned_tasks, initial_available_resources, working_resources, trds, mean_durations, resource_pool)

    print('Delta:', str(i))
    print(trds)
    print(log)
    print('Time: ', time)

[t1, t2] ['Ra', 'Rb'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {(t1, 'Ra'): 3, (t1, 'Rb'): 15, (t1, 'Rd'): 21, (t2, 'Ra'): 7, (t2, 'Rc'): 7, (t2, 'Rd'): 22} {t1: 13.0, t2: 12.0} {} 0
[t2] ['Rb', 'Rc'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {(t2, 'Ra'): 7, (t2, 'Rc'): 7, (t2, 'Rd'): 22} {t1: 13.0, t2: 12.0} {'Ra': (0, 3)} 2
[] ['Rb', 'Ra'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {} {t1: 13.0, t2: 12.0} {'Rc': (2, 7)} 3
[] ['Rb', 'Ra', 'Rc'] {'t1': ['Ra', 'Rb'], 't2': ['Ra', 'Rc']} {} {t1: 13.0, t2: 12.0} {} 9
Delta: 1
{(t1, 'Ra'): 3, (t1, 'Rb'): 15, (t1, 'Rd'): 21, (t2, 'Ra'): 7, (t2, 'Rc'): 7, (t2, 'Rd'): 22}
0
Suggested allocation: [(t1, 'Ra'), (t2, 'Ra')]
Real allocation: [(t1, 'Ra')]
[t2], ['Rb'], {(t2, 'Ra'): 7, (t2, 'Rc'): 7, (t2, 'Rd'): 22}, {'Ra': (0, 3)}
Rc spawned
2
Suggested allocation: [(t2, 'Rc')]
Real allocation: [(t2, 'Rc')]
[], ['Rb'], {}, {'Ra': (0, 3), 'Rc': (2, 7)}
3
Suggested allocation: []
Real allocation: []
[], ['Rb', 'Ra'], {}, {'Rc': (2, 7)}
9
Suggested alloc

In [12]:
for i,j in [(t1, 'Ra'), (t2, 'Ra')]:
    print(i,j)

t1 Ra
t2 Ra
